In [2]:
import sqlite3
import pandas as pd

In [3]:
# 'mydatabase.db' نام فایل پایگاه داده ماست. می‌تونه هر چیزی باشه.
conn = sqlite3.connect('mydatabase.db')

# یک شیء 'cursor' ایجاد می‌کنیم که برای اجرای دستورات SQL استفاده میشه.
cursor = conn.cursor()

print("به پایگاه داده متصل شدیم!")

به پایگاه داده متصل شدیم!


In [4]:
# جدول Customers
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID INTEGER PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    City VARCHAR(50),
    Country VARCHAR(50),
    RegistrationDate DATE
);
''')

# جدول Products
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName VARCHAR(100),
    Category VARCHAR(50),
    UnitPrice DECIMAL(10, 2),
    UnitsInStock INTEGER
);
''')

# جدول Orders
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    OrderDate DATE,
    TotalAmount DECIMAL(10, 2),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

# جدول OrderDetails
cursor.execute('''
CREATE TABLE IF NOT EXISTS OrderDetails (
    OrderDetailID INTEGER PRIMARY KEY,
    OrderID INTEGER,
    ProductID INTEGER,
    Quantity INTEGER,
    ItemPrice DECIMAL(10, 2),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
''')

conn.commit() # تغییرات رو ذخیره کن
print("جدول‌ها ایجاد شدند (اگر وجود نداشتند).")

جدول‌ها ایجاد شدند (اگر وجود نداشتند).


In [5]:
# وارد کردن داده‌ها به Customers
customers_data = [
    (1, 'علی', 'احمدی', 'ali@example.com', 'تهران', 'ایران', '2023-01-15'),
    (2, 'سارا', 'محمدی', 'sara@example.com', 'اصفهان', 'ایران', '2023-02-20'),
    (3, 'جان', 'اسمیت', 'john@example.com', 'New York', 'USA', '2022-11-10'),
    (4, 'مریم', 'حسینی', 'maryam@example.com', 'تهران', 'ایران', '2024-03-01'),
    (5, 'لیلا', 'کریمی', None, 'شیراز', 'ایران', '2023-07-22'),
    (6, 'پاول', 'نیکولاس', 'paul@example.com', 'Berlin', 'Germany', '2023-04-05')
]
cursor.executemany("INSERT OR IGNORE INTO Customers VALUES (?, ?, ?, ?, ?, ?, ?)", customers_data) # IGNORE برای جلوگیری از خطای تکراری بودن PK

# وارد کردن داده‌ها به Products
products_data = [
    (101, 'لپ تاپ', 'الکترونیک', 1200.00, 50),
    (102, 'گوشی موبایل', 'الکترونیک', 800.00, 120),
    (103, 'تی‌شرت', 'لباس', 25.00, 300),
    (104, 'کتاب داستان', 'کتاب', 15.50, 500),
    (105, 'هدفون بی‌سیم', 'الکترونیک', 150.00, 80),
    (106, 'شلوار جین', 'لباس', 60.00, 150)
]
cursor.executemany("INSERT OR IGNORE INTO Products VALUES (?, ?, ?, ?, ?)", products_data)

# وارد کردن داده‌ها به Orders
orders_data = [
    (1001, 1, '2024-01-20', 1225.00),
    (1002, 3, '2024-01-25', 800.00),
    (1003, 1, '2024-02-10', 150.00),
    (1004, 4, '2024-03-05', 25.00),
    (1005, 2, '2024-04-12', 1200.00),
    (1006, 6, '2024-05-01', 60.00),
    (1007, 4, '2024-06-15', 800.00),
    (1008, 5, '2024-06-20', 15.50)
]
cursor.executemany("INSERT OR IGNORE INTO Orders VALUES (?, ?, ?, ?)", orders_data)

# وارد کردن داده‌ها به OrderDetails
order_details_data = [
    (1, 1001, 101, 1, 1200.00),
    (2, 1001, 103, 1, 25.00),
    (3, 1002, 102, 1, 800.00),
    (4, 1003, 105, 1, 150.00),
    (5, 1004, 103, 1, 25.00),
    (6, 1005, 101, 1, 1200.00),
    (7, 1006, 106, 1, 60.00),
    (8, 1007, 102, 1, 800.00),
    (9, 1008, 104, 1, 15.50)
]
cursor.executemany("INSERT OR IGNORE INTO OrderDetails VALUES (?, ?, ?, ?, ?)", order_details_data)

conn.commit()
print("داده‌های نمونه وارد شدند (اگر وجود نداشتند).")

داده‌های نمونه وارد شدند (اگر وجود نداشتند).


In [6]:
# مثال 1: تمام مشتریان رو انتخاب کن
query1 = "SELECT * FROM Customers;"
df_customers = pd.read_sql_query(query1, conn)
print("تمام مشتریان:")
print(df_customers)
print("-" * 30)

تمام مشتریان:
   CustomerID FirstName LastName               Email      City  Country  \
0           1       علی    احمدی     ali@example.com     تهران    ایران   
1           2      سارا    محمدی    sara@example.com    اصفهان    ایران   
2           3       جان    اسمیت    john@example.com  New York      USA   
3           4      مریم    حسینی  maryam@example.com     تهران    ایران   
4           5      لیلا    کریمی                None     شیراز    ایران   
5           6      پاول  نیکولاس    paul@example.com    Berlin  Germany   

  RegistrationDate  
0       2023-01-15  
1       2023-02-20  
2       2022-11-10  
3       2024-03-01  
4       2023-07-22  
5       2023-04-05  
------------------------------


In [7]:
# مثال 2: محصولات الکترونیک
query2 = "SELECT ProductName, UnitPrice FROM Products WHERE Category = 'الکترونیک';"
df_electronics = pd.read_sql_query(query2, conn)
print("محصولات الکترونیک:")
print(df_electronics)
print("-" * 30)

محصولات الکترونیک:
    ProductName  UnitPrice
0        لپ تاپ       1200
1   گوشی موبایل        800
2  هدفون بی‌سیم        150
------------------------------


In [8]:
# مثال 3: جوین ساده - سفارشات همراه با نام مشتری
query3 = """
SELECT
    o.OrderID,
    c.FirstName || ' ' || c.LastName AS CustomerFullName, -- ترکیب نام و نام خانوادگی
    o.OrderDate,
    o.TotalAmount
FROM Orders AS o
INNER JOIN Customers AS c ON o.CustomerID = c.CustomerID;
"""
df_orders_customers = pd.read_sql_query(query3, conn)
print("سفارشات با نام مشتری:")
print(df_orders_customers)
print("-" * 30)

سفارشات با نام مشتری:
   OrderID CustomerFullName   OrderDate  TotalAmount
0     1001        علی احمدی  2024-01-20       1225.0
1     1002        جان اسمیت  2024-01-25        800.0
2     1003        علی احمدی  2024-02-10        150.0
3     1004       مریم حسینی  2024-03-05         25.0
4     1005       سارا محمدی  2024-04-12       1200.0
5     1006     پاول نیکولاس  2024-05-01         60.0
6     1007       مریم حسینی  2024-06-15        800.0
7     1008       لیلا کریمی  2024-06-20         15.5
------------------------------


In [9]:

# مثال 4: استفاده از GROUP BY و HAVING
query4 = """
SELECT
    c.City,
    COUNT(c.CustomerID) AS NumberOfCustomers
FROM Customers AS c
GROUP BY c.City
HAVING COUNT(c.CustomerID) > 1; -- شهرهایی با بیش از 1 مشتری
"""
df_city_counts = pd.read_sql_query(query4, conn)
print("شهرهایی با بیش از یک مشتری:")
print(df_city_counts)
print("-" * 30)

شهرهایی با بیش از یک مشتری:
    City  NumberOfCustomers
0  تهران                  2
------------------------------


# مسئله شماره ۱: لیست کامل مشتریان
## صورت مسئله:
### یک لیست از نام و نام خانوادگی کامل مشتریان رو همراه با شهر و کشور اون‌ها به دست بیارید. لیست نهایی باید بر اساس کشور (صعودی) و سپس نام خانوادگی (صعودی) مرتب شده باشه.

In [10]:
question1 = '''
select 
    Country , 
    City,
    LastName,
    FirstName,
    FirstName || ' ' || LastName as FullName
from customers
order by
    Country asc,
    LastName asc
'''
df_customers = pd.read_sql_query(question1 , conn)
print(df_customers)


   Country      City LastName FirstName      FullName
0  Germany    Berlin  نیکولاس      پاول  پاول نیکولاس
1      USA  New York    اسمیت       جان     جان اسمیت
2    ایران     تهران    احمدی       علی     علی احمدی
3    ایران     تهران    حسینی      مریم    مریم حسینی
4    ایران    اصفهان    محمدی      سارا    سارا محمدی
5    ایران     شیراز    کریمی      لیلا    لیلا کریمی


# مسئله شماره ۲: محصولات الکترونیک
## صورت مسئله:
### نام محصولات الکترونیک (Category = 'الکترونیک') رو به همراه قیمت واحد اون‌ها به دست بیارید. لیست نهایی باید بر اساس گران‌ترین محصول (قیمت بالا به پایین) مرتب شده باشه.

In [14]:
question2 = '''
select 
    Category,
    UnitPrice,
    ProductName
from Products
where category = 'الکترونیک'
order by UnitPrice desc;'''
df_category = pd.read_sql_query(question2 , conn)
print(df_category)

    Category  UnitPrice   ProductName
0  الکترونیک       1200        لپ تاپ
1  الکترونیک        800   گوشی موبایل
2  الکترونیک        150  هدفون بی‌سیم
